# Fit Neural Networks to ecotoxicological data

Import modules and prepare data set

In [5]:
import sys
genpath = 'D:/Code'
if genpath not in sys.path:
    sys.path.append(genpath)

import DataSciPy
from sklearn.linear_model import Perceptron
import pickle
import pandas as pd
import src.general_helper as genH

Load the data set and split the `fingerprint`-column to one column for each character.

In [40]:
final_db = pd.read_csv('data/processed/final_db_update.csv')
print(final_db.columns)
fp = pd.DataFrame(final_db.fingerprint.apply(genH.splt_str))
fp = pd.DataFrame(fp.fingerprint.tolist(), index=fp.index)
final_db = final_db.drop(columns=['fingerprint']).join(fp).drop(columns=['test_cas'])

Index(['test_cas', 'species', 'conc1_type', 'exposure_type',
       'obs_duration_mean', 'conc1_mean', 'class', 'tax_order', 'family',
       'genus', 'fingerprint'],
      dtype='object')


Prepare the dataset object

In [41]:
# These are the names of the columns that should be encoded
encode_these = ['species','conc1_type','exposure_type','class','tax_order',
                'family','genus','obs_duration_mean']
final_db = genH.binary_score(final_db)
dummy = DataSciPy.Dataset()
dummy.setup_data(X=final_db.drop(columns=['score']),
                 y=final_db.loc[:,['score']],
                 split_test=0.3)
dummy.encode_categories(variables=encode_these)

There are 8566 datapoints with label 0 and 14761 datapoints with label 1
(16328, 889)


Fit perceptron to obtain a baseline model

In [42]:
per = Perceptron(tol=1e-3, random_state=0)
per.fit(dummy.X_train, dummy.y_train.iloc[:,0])
per.score(dummy.X_test, dummy.y_test.iloc[:,0])

0.39005572224603513